# Homework - Grover MaxCut

The places where you have enter code are marked with `# YOUR CODE HERE`.

In [ ]:
import cirq
from cirq import H, X, Y, Z, CX, inverse

## Question 1 (4 points)

Write a function, `oracle010`, that implements an oracle that marks the state $|010 \rangle$. The function `oracle010` has

* input: `qq`, a 3-qubit register 
* returns: `None`

The function should append a sequence of gates to `qq` to mark the state $|010\rangle$ only. Don't append any measurements to `qq`.

To help you test the function, we have provided the `grover_diffusion` and `grover` functions.

In [ ]:
def oracle010(qq):
     # Flip the first and third qubits to turn |010> into |111>
    yield X(qq[0]), X(qq[2])
    # Apply CCZ gate (mark |111>)
    yield Z.controlled(2)(qq[0], qq[1], qq[2])
    # Flip the first and third qubits back
    yield X(qq[0]), X(qq[2])

In [ ]:
# visualize your implemented gates
qqTest = cirq.LineQubit.range(3)
circuit = cirq.Circuit()
circuit.append(oracle010(qqTest))
circuit

In [ ]:
# To check your solution, we need some to implement grover
def grover_diffusion(qq,n):
    yield H.on_each(*qq)
    yield X.on_each(*qq)
    yield Z(qq[n-1]).controlled_by(*(qq[0:n-1]))
    yield X.on_each(*qq)
    yield H.on_each(*qq)

In [ ]:
def grover(trials_number):
    n=3
    qq = cirq.LineQubit.range(n)
    circuit = cirq.Circuit()
    circuit.append(H.on_each(*qq))  

    for i in range(2):
        circuit.append(oracle010(qq))
        circuit.append(grover_diffusion(qq,n))
    circuit.append(cirq.measure(*qq, key='result'))

    # determine the statistics of the measurements
    s = cirq.Simulator() 
    samples = s.run(circuit, repetitions=trials_number)

    def bitstring(bits):
        return "".join(str(int(b)) for b in bits)

    counts = samples.histogram(key="result",fold_func=bitstring)
    print(counts)
    return counts.get('010')

In [ ]:
# run grover to test if your function gives the right answer
grover(100)

In [ ]:
# hidden tests in this cell will be used for grading.

## Question 2 (6 points)

Graph $G$ has 5 vertices and 6 edges: (0,3), (0,4), (1,3), (1,4), (2,3), (2,4).

Write an oracle for the graph $G$ to check whether it admits a valid 2-coloring. 

The function `oracle2` has

* input: `qq`, a 12-qubit register 
* returns: `None`

The function should append only a sequence of gates to `qq`. It should not append any measurements to `qq`.

Use qubits 0-4 for the vertices, 5-10 for the edges and 11 as the ancilla. 

You can test the oracle with the provided `grover_diffusion`, `grover` and `oracle_computation2` functions.

In [ ]:
def oracle2(qq):
    # Qubit assignments:
    # Vertices: 0-4
    # Edges: 5-10 (6 edges)
    # Ancilla: 11
    
    edges = [(0,3), (0,4), (1,3), (1,4), (2,3), (2,4)]
    
    # Check each edge and store the result in edge qubits
    for i, (u, v) in enumerate(edges):
        edge_qubit = 5 + i
        # The edge qubit should be |1> if u and v are different
        yield CX(qq[u], qq[edge_qubit])
        yield CX(qq[v], qq[edge_qubit])
    
    # Flip the ancilla if all edge qubits are |1>
    yield X(qq[11]).controlled_by(*[qq[i] for i in range(5, 11)])
    
    # Uncompute the edge qubits to restore them to |0>
    for i, (u, v) in enumerate(edges[::-1]):  # reverse order to properly uncompute
        edge_qubit = 5 + (len(edges) - 1 - i)
        yield CX(qq[v], qq[edge_qubit])
        yield CX(qq[u], qq[edge_qubit])

In [ ]:
# We need some code so you can check your solution
def oracle_computation2(qq):
    yield oracle2(qq)
    yield Z(qq[11])
    yield inverse(oracle2(qq))  

In [ ]:
def grover2(trials_number):    
    import cirq
    from cirq import X, H, Z, inverse, CX
    s = cirq.Simulator()

    qq = cirq.LineQubit.range(12)
    n=5
    
    circuit = cirq.Circuit()
    circuit.append(H.on_each(*(qq[0:n])))
    for i in range(2):
        circuit.append(oracle_computation2(qq))
        circuit.append(grover_diffusion(qq,n))

    circuit.append(cirq.measure(*(qq[0:n]), key='result'))

    # determine the statistics of the measurements
    samples = s.run(circuit, repetitions=trials_number)
    result = samples.measurements["result"]

    def bitstring(bits):
        return "".join(str(int(b)) for b in bits)

    counts = samples.histogram(key="result",fold_func=bitstring)
    return counts

In [ ]:
#You can use this cell to test your solution
shots=1000
grover2(shots)

In [ ]:
# hidden tests in this cell will be used for grading.

## Question 3 (10 points)

Graph $G$ has 4 vertices and 5 edges: (0,1), (0,2), (0,3), (1,2), (1,3)

Write an oracle for the graph $G$ to check whether there exists a coloring with at least 4 edges connecting vertices with different colors.

The function `oracle3` has

* input: `qq`, a 13-qubit register 
* returns: `None`

The function should append only a sequence of gates to `qq`. It should not append any measurements to `qq`.

Use qubits 
- 0-3 for the vertices,
- 4-8 for the edges,
- 9-11 for the addition (remember we need three qubits here for addition unlike the last question), and
- 12 as the ancilla.

You can test the oracle with the provided `grover_diffusion`, `grover` and `oracle_computation3` functions.

In [ ]:
def oracle3(qq):
    edges = [(0, 1), (0, 2), (0, 3), (1, 2), (1, 3)]
    
    # Step 1: Encode edge validities (XOR results into edge qubits 4-8)
    for i, (u, v) in enumerate(edges):
        edge_qubit = 4 + i
        yield CX(qq[u], qq[edge_qubit])
        yield CX(qq[v], qq[edge_qubit])
    
    # Step 2: Count valid edges into addition qubits 9-11 (3-bit counter)
    for edge_idx in range(5):
        e = 4 + edge_idx  # Current edge qubit
        
        # Increment counter by 1 if edge is valid (controlled by e)
        # First, handle the least significant bit (qubit 9)
        yield CX(qq[e], qq[9])
        
        # Now handle the carry to qubit 10 if both e and qubit 9 were 1
        yield CCX(qq[e], qq[9], qq[10])
        
        # Handle carry to qubit 11 if e, qubit 9, and qubit 10 are all 1
        yield CCX(qq[e], qq[10], qq[11])
    
    # Step 3: Check if count >=4 (MSB qubit 11 is |1>)
    yield CX(qq[11], qq[12])
    
    # Step 4: Uncompute the counter
    for edge_idx in reversed(range(5)):
        e = 4 + edge_idx
        
        # Uncompute carry to qubit 11
        yield CCX(qq[e], qq[10], qq[11])
        
        # Uncompute carry to qubit 10
        yield CCX(qq[e], qq[9], qq[10])
        
        # Uncompute LSB
        yield CX(qq[e], qq[9])
    
    # Step 5: Uncompute edge validities
    for i, (u, v) in enumerate(reversed(edges)):
        edge_qubit = 4 + (len(edges) - 1 - i)
        yield CX(qq[v], qq[edge_qubit])
        yield CX(qq[u], qq[edge_qubit])

In [ ]:
# We need some code so you can check your solution
def oracle_computation3(qq):
    yield oracle3(qq)
    yield Z(qq[12])
    yield inverse(oracle3(qq))  

In [ ]:
import cirq
from cirq import X, H, Z, inverse, CX, CCX
    
def grover3(trials_number):    
    s = cirq.Simulator()

    qq = cirq.LineQubit.range(13)
    n=4

    circuit = cirq.Circuit()
    circuit.append(H.on_each(*(qq[0:n])))
    for i in range(2):
        circuit.append(oracle_computation3(qq))
        circuit.append(grover_diffusion(qq,n))

    circuit.append(cirq.measure(*(qq[0:n]), key='result'))

    # determine the statistics of the measurements
    samples = s.run(circuit, repetitions=trials_number)
    result = samples.measurements["result"]

    def bitstring(bits):
        return "".join(str(int(b)) for b in bits)

    counts = samples.histogram(key="result",fold_func=bitstring)
    return counts

In [ ]:
#You can use this cell to test your solution
shots=1000
grover3(shots)

In [ ]:
# hidden tests in this cell will be used for grading.